In [30]:
target, seed = 'IC50_nM', 42

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import *
from sklearn.ensemble import *
from sklearn.metrics import * # Score Criteria
import lightgbm as lgb

# Visualization
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [138]:
# Regularization, Estimator
from sklearn.linear_model import Lasso,ElasticNet,Ridge
from sklearn.model_selection import cross_val_score, validation_curve, GridSearchCV
from sklearn.metrics import * # RMSLE, RMSE, r2, accuracy_score

from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Scaler, Encoder, Regulariation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
from sklearn.linear_model import Lasso, Ridge, ElasticNet

In [198]:
X_train, y_train = pd.read_csv('/content/X_train.csv'), pd.read_csv('/content/y_train')
X_test, submission = pd.read_csv('/content/X_test.csv'), pd.read_csv('/content/sample_submission.csv')
train = pd.concat([X_train, y_train], axis=1)

# 각 list의 값들이 같아야함.
[len(X_train), len(y_train)], [len(X_test), len(submission)], [len(X_train.columns), len(X_test.columns)]

([1952, 1952], [113, 113], [132, 132])

In [204]:
smiles = pd.read_csv('/content/train.csv')['Smiles']

# Preparation for EDA

In [199]:
# All features are numerical
print(len(X_train.columns) - len(X_train.select_dtypes(exclude=object).columns))

0


In [200]:
X_train.head().T

,0,1,2,3,4
MaxEStateIndex,14.635817,14.673248,14.663674,14.610375,14.011531
MinEStateIndex,-0.828882,-1.585128,-1.596212,-2.432700,-0.859426
MaxAbsEStateIndex,14.635817,14.673248,14.663674,14.610375,14.011531
MinAbsEStateIndex,0.021978,0.205025,0.195978,0.119300,0.000644
qed,0.059187,0.479279,0.466884,0.448013,0.045219
...,...,...,...,...,...
fr_ether,5.000000,0.000000,1.000000,0.000000,2.000000
fr_nitrile,0.000000,0.000000,0.000000,0.000000,0.000000
fr_piperdine,0.000000,1.000000,0.000000,0.000000,0.000000
fr_pyridine,1.000000,0.000000,0.000000,0.000000,1.000000


In [201]:
def remove_low_cor_cols(df, target_column, threshold=0.1):
    # target_column과의 상관계수 계산
    correlations = df.corr()[target_column].abs()

    low_corr_columns = correlations[correlations < threshold].index

    # low_corr_columns를 데이터프레임에서 제거
    df_filtered = df.drop(columns=low_corr_columns)

    return df_filtered

train = remove_low_cor_cols(train, target, threshold=0.1)
X_train, y_train = train.drop(columns=[target]), train[target]

print(len(train.columns), len(X_train.columns))

65 64


In [202]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1952 entries, 0 to 1951
Data columns (total 64 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MinEStateIndex           1952 non-null   float64
 1   MolWt                    1952 non-null   float64
 2   HeavyAtomMolWt           1952 non-null   float64
 3   ExactMolWt               1952 non-null   float64
 4   NumValenceElectrons      1952 non-null   int64  
 5   BCUT2D_CHGHI             1952 non-null   float64
 6   BCUT2D_CHGLO             1952 non-null   float64
 7   BCUT2D_LOGPLOW           1952 non-null   float64
 8   BalabanJ                 1952 non-null   float64
 9   BertzCT                  1952 non-null   float64
 10  Chi0                     1952 non-null   float64
 11  Chi0n                    1952 non-null   float64
 12  Chi0v                    1952 non-null   float64
 13  Chi1                     1952 non-null   float64
 14  Chi1n                   

In [203]:
train.corr()[target].abs().sort_values(ascending=False)

,IC50_nM
IC50_nM,1.000000
NumAromaticHeterocycles,0.256691
SMR_VSA3,0.247292
BCUT2D_LOGPLOW,0.246095
NumAromaticCarbocycles,0.233334
...,...
HallKierAlpha,0.108043
NumSaturatedRings,0.107994
VSA_EState7,0.102991
SMR_VSA7,0.102473


# Visualization

# Model Baseline

In [165]:
X_train, y_train = train.drop(columns=[target]), train[target]

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Model_01. LGBMRegressor

In [142]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error


lgb_model = LGBMRegressor(learning_rate=0.1, n_estimators=3000, random_seed=42)
lgb_model.fit(X_train, y_train )

lgb_pred= lgb_model.predict(X_test)

mean_squared_error(y_test, y_pred)**0.5

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001175 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10727
[LightGBM] [Info] Number of data points in the train set: 1464, number of used features: 64
[LightGBM] [Info] Start training from score 657.481212


2045.0068606697116

# Model_02. MLP Network

In [194]:
X_cols = X_train.columns

for col in X_test.columns :
  if col not in X_cols :
    X_test = X_test.drop([col], axis=1)

len(X_test.columns)

64

In [195]:
from sklearn.neural_network import MLPRegressor

MLP_model = MLPRegressor(hidden_layer_sizes=[500, 500, 500, 500, 500],
                         max_iter=10_000, random_state=seed)

MLP_model.fit(X_train, y_train)
MLP_pred = MLP_model.predict(X_test)

In [177]:
mean_squared_error(y_test, MLP_pred)**0.5

1890.184946201814

In [197]:
MLP_submission = pd.DataFrame({'ID':submission['ID'], 'IC50_nM': MLP_pred * 0.205})
MLP_submission.to_csv('MLP_submission.csv', index=False)